In [1]:
import pandas as pd
import numpy as np
import json
import datetime
from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
with open(r"C:\Users\Nick\OneDrive\Desktop\model\news.json", "r", encoding="utf_8_sig") as news_json:
    data = json.loads(news_json.read())

news_types = set()

for i in data:
    news_types.add(str(i.get('id'))[-3:])

print(news_types)

"""
073 - новости, 050 - новости мэра. news_types хотела определить 
какие вообще нам выдали типы, так как есть еще другие.
"""

df = pd.read_excel(r"C:\Users\Nick\OneDrive\Desktop\model\dataset_news_1.xlsx")


def get_news_id_from_url(url: str) -> int:
    """
    Вычленить id из url
    """
    parts = url.split('/')
    try:
        if parts[-2] == '9468':
            print(url)
        return int(parts[-2])
    except Exception as err:
        print(url)
        for part in parts:
            if '073' in part:  # Опытным путем выявлено, что битые урлы
                # только для типа 073, поэтому просто решила вытащить такие
                return int(part)
        print("it is not news url ", url)
        return 0

{'073', '050'}


In [4]:
with open(r"C:\Users\Nick\OneDrive\Desktop\model\news.json", "r", encoding="utf_8_sig") as news_json:
    data = json.loads(news_json.read())

news_types = set()

for i in data:
    news_types.add(str(i.get('id'))[-3:])


df = pd.read_excel(r"C:\Users\Nick\OneDrive\Desktop\model\dataset_news_1.xlsx")


def get_news_id_from_url(url: str) -> int:

    parts = url.split('/')
    try:
        if parts[-2] == '9468':
            print(url)
        return int(parts[-2])
    except Exception as err:
        print(url)
        for part in parts:
            if '073' in part:  
                return int(part)
        print("it is not news url ", url)
        return 0

In [5]:
def get_news_type(news_id: int) -> str:
   
    if str(news_id)[-3:] == '073':
        return "news"
    if str(news_id)[-3:] == '050':
        return "major"
    else:
        print(news_id)
        return "error"

In [6]:
df['news_id'] = df['url_clean'].apply(get_news_id_from_url)
df['news_type'] = df['news_id'].apply(get_news_type)

df['news_type'].value_counts()

news_ids_from_ds = set(df['news_id'].unique())
news_ids_from_json = set([i.get('id') for i in data])

mos.ru/news/item/89421073/ /
mos.ru/news/item/9468/
mos.ru/news/item/94670073/ /
mos.ru/news/item/94501073/душ/
mos.ru/news/item/89957073/ Их/
mos.ru/news/item/94852073/%5c/
mos.ru/news/item/94479073/ (https:/app.aif.ru/owa/redir.aspx/
mos.ru/news/item/94792073/ /
mos.ru/news/item/94897073/+/
mos.ru/news/item/94953073/ /
mos.ru/news/item/91919073/-/
9514707
9468


In [7]:
df_json = pd.read_json(r"C:\Users\Nick\OneDrive\Desktop\model\news.json", encoding="utf_8_sig")  # Закинем json в df
df_json["news_type"] = df_json['id'].apply(get_news_type)
df_json["news_type"].value_counts()

news     5729
major     825
Name: news_type, dtype: int64

In [8]:
df_json = pd.read_json(r"C:\Users\Nick\OneDrive\Desktop\model\news.json", encoding="utf_8_sig") 
df_json["news_type"] = df_json['id'].apply(get_news_type)
df_json["news_type"].value_counts()

with open(r"C:\Users\Nick\OneDrive\Desktop\model\districts.json", "r", encoding="utf_8_sig") as file:
    data_districts = json.loads(file.read())

districts = list()
for a in data_districts["items"]:
    for d in a['areas']:
        d['area_title'] = a['title']
        d['area_local_id'] = a['local_id']
        districts.append(d)
    if not a['areas']:
        districts.append({
            "id": a['id'],
            "title": a['title'],
            "local_id": a['local_id'],
            "district_id": None,
            "site": None,
            "population": None,
            "square": None,
            "manager_id": None,
            "title_en": a['title_en'],
            'area_title': a['title'],
            'area_local_id': a['local_id']
        })

districts_df = pd.DataFrame(districts)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26446 entries, 0 to 26445
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  26446 non-null  datetime64[ns]
 1   url_clean  26446 non-null  object        
 2   user_id    26446 non-null  int64         
 3   news_id    26446 non-null  int64         
 4   news_type  26446 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.0+ MB


In [10]:
merged = df.merge(df_json, 
                  left_on='news_id', 
                  right_on='id')
len(df.user_id.unique())

239

In [11]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26442 entries, 0 to 26441
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date_time                26442 non-null  datetime64[ns]
 1   url_clean                26442 non-null  object        
 2   user_id                  26442 non-null  int64         
 3   news_id                  26442 non-null  int64         
 4   news_type_x              26442 non-null  object        
 5   id                       26442 non-null  int64         
 6   title                    26442 non-null  object        
 7   importance               21599 non-null  object        
 8   published_at             26442 non-null  datetime64[ns]
 9   created_at               26442 non-null  datetime64[ns]
 10  updated_at               26442 non-null  datetime64[ns]
 11  is_deferred_publication  22616 non-null  float64       
 12  status                   26442 n

In [12]:
show_difference = pd.to_datetime(merged['date_time']).sub(merged['published_at'])
days = show_difference.dt.days
merged['published/read'] = days
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26442 entries, 0 to 26441
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date_time                26442 non-null  datetime64[ns]
 1   url_clean                26442 non-null  object        
 2   user_id                  26442 non-null  int64         
 3   news_id                  26442 non-null  int64         
 4   news_type_x              26442 non-null  object        
 5   id                       26442 non-null  int64         
 6   title                    26442 non-null  object        
 7   importance               21599 non-null  object        
 8   published_at             26442 non-null  datetime64[ns]
 9   created_at               26442 non-null  datetime64[ns]
 10  updated_at               26442 non-null  datetime64[ns]
 11  is_deferred_publication  22616 non-null  float64       
 12  status                   26442 n

In [13]:
final_df = merged.drop(['importance', 'is_deferred_publication', 'status', 'ya_rss', 'active_from',
                       'active_to', 'search', 'display_image', 'icon_id', 'canonical_url', 'canonical_updated_at',
                       'is_powered', 'has_image', 'attach', 'active_from_timestamp', 'active_to_timestamp',
                       'image', 'counter', 'preview_text', 'images'],
                      axis=1)
final_df

,date_time,url_clean,user_id,news_id,news_type_x,id,title,published_at,created_at,updated_at,oiv_id,label,date,has_district,date_timestamp,tags,theme_id,theme_ids,themes,spheres,sphere,kind,is_oiv_publication,organizations,updated_at_timestamp,created_at_timestamp,territory_area_id,territory_district_id,full_text,url,preview,text,promo,news_type_y,published/read
0,2021-08-01 18:51:19,mos.ru/news/item/94006073/,1,94006073,news,94006073,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00,2021-07-30 07:29:53,2021-08-01 09:03:12,0.0,None,2021-08-01 09:04:00,1,1627797840,"[{'id': 144217, 'title': 'архитектура', 'creat...",115287.0,"[115287, 132287]","[{'id': 115287, 'title': 'Интересная Москва', ...","[{'id': 3299, 'title': 'Культура', 'special': ...","{'id': 3299, 'title': 'Культура', 'special': 0...","{'id': 'article', 'title': 'Новости', 'type': 1}",0.0,[],1.627798e+09,1.627619e+09,1501.0,1500.0,<p>Если во время прогулки по Москве внимательн...,/news/item/94006073/,NaN,NaN,NaN,news,0
1,2021-08-04 13:08:19,mos.ru/news/item/94006073/,2,94006073,news,94006073,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00,2021-07-30 07:29:53,2021-08-01 09:03:12,0.0,None,2021-08-01 09:04:00,1,1627797840,"[{'id': 144217, 'title': 'архитектура', 'creat...",115287.0,"[115287, 132287]","[{'id': 115287, 'title': 'Интересная Москва', ...","[{'id': 3299, 'title': 'Культура', 'special': ...","{'id': 3299, 'title': 'Культура', 'special': 0...","{'id': 'article', 'title': 'Новости', 'type': 1}",0.0,[],1.627798e+09,1.627619e+09,1501.0,1500.0,<p>Если во время прогулки по Москве внимательн...,/news/item/94006073/,NaN,NaN,NaN,news,3
2,2021-08-29 12:40:07,mos.ru/news/item/94006073/,3,94006073,news,94006073,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00,2021-07-30 07:29:53,2021-08-01 09:03:12,0.0,None,2021-08-01 09:04:00,1,1627797840,"[{'id': 144217, 'title': 'архитектура', 'creat...",115287.0,"[115287, 132287]","[{'id': 115287, 'title': 'Интересная Москва', ...","[{'id': 3299, 'title': 'Культура', 'special': ...","{'id': 3299, 'title': 'Культура', 'special': 0...","{'id': 'article', 'title': 'Новости', 'type': 1}",0.0,[],1.627798e+09,1.627619e+09,1501.0,1500.0,<p>Если во время прогулки по Москве внимательн...,/news/item/94006073/,NaN,NaN,NaN,news,28
3,2021-08-02 09:04:55,mos.ru/news/item/94006073/,6,94006073,news,94006073,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00,2021-07-30 07:29:53,2021-08-01 09:03:12,0.0,None,2021-08-01 09:04:00,1,1627797840,"[{'id': 144217, 'title': 'архитектура', 'creat...",115287.0,"[115287, 132287]","[{'id': 115287, 'title': 'Интересная Москва', ...","[{'id': 3299, 'title': 'Культура', 'special': ...","{'id': 3299, 'title': 'Культура', 'special': 0...","{'id': 'article', 'title': 'Новости', 'type': 1}",0.0,[],1.627798e+09,1.627619e+09,1501.0,1500.0,<p>Если во время прогулки по Москве внимательн...,/news/item/94006073/,NaN,NaN,NaN,news,1
4,2021-08-02 17:16:23,mos.ru/news/item/94006073/,11,94006073,news,94006073,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00,2021-07-30 07:29:53,2021-08-01 09:03:12,0.0,None,2021-08-01 09:04:00,1,1627797840,"[{'id': 144217, 'title': 'архитектура', 'creat...",115287.0,"[115287, 132287]","[{'id': 115287, 'title': 'Интересная Москва', ...","[{'id': 3299, 'title': 'Культура', 'special': ...","{'id': 3299, 'title': 'Культура', 'special': 0...","{'id': 'article', 'title': 'Новости', 'type': 1}",0.0,[],1.627798e+09,1.627619e+09,1501.0,1500.0,<p>Если во время прогулки по Москве внимательн...,/news/item/94006073/,NaN,NaN,NaN,news,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26437,2021-08-26 12:16:23,mos.ru/news/item/65753073/,275,65753073,news,65753073,"Концерты, зумба и ходьба: что подготовили цент...",2019-11-24 09:12:00,2019-11-23 13:39:00,2020-06-11 19:33:47,0.0,None,2019-11-24 09:12:00,1,1574575920,"[{'i

In [14]:
final_df['sphere_id'] = final_df.sphere.map(lambda row: row.get('id'))

In [15]:
final_df['title_age'] = (pd.Timestamp.now() - final_df['published_at']).dt.days

In [16]:
final_df['age_param'] = (1 / final_df['title_age']) * 10000

In [17]:
users, items, interactions = final_df.user_id.nunique(), final_df.id.nunique(), final_df.shape[0]
print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  239
# items:  5809
# interactions:  26442


In [18]:
26442 / (users*items)


0.019045615986159118

In [19]:
final_df['date_time'].dt.day.min()
final_df = final_df[['user_id', 'news_id', 'date_time', 'age_param', 'title', 'published_at']]
final_df.head()

,user_id,news_id,date_time,age_param,title,published_at
0,1,94006073,2021-08-01 18:51:19,119.047619,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00
1,2,94006073,2021-08-04 13:08:19,119.047619,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00
2,3,94006073,2021-08-29 12:40:07,119.047619,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00
3,6,94006073,2021-08-02 09:04:55,119.047619,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00
4,11,94006073,2021-08-02 17:16:23,119.047619,Дома-порталы: где в Москве можно почувствовать...,2021-08-01 09:04:00


In [20]:
result = final_df.groupby('user_id')['news_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[94006073, 94000073, 94062073, 94063073, 93893..."
1,2,"[94006073, 94662073, 94679073, 94115073, 92326..."
2,3,"[94006073, 94108073, 94057073, 94290073, 75470..."
3,4,"[94647073, 94449073, 94669073, 94679073, 75730..."
4,5,"[94000073, 94098073, 94366073, 94483073, 94381..."
5,6,"[94006073, 94063073, 94098073, 94108073, 94132..."
6,7,"[94098073, 94060073, 94249073, 94178073, 94621..."
7,8,"[94249073, 94381073, 94581073, 93028073, 93846..."
8,9,"[94642073, 94572073, 94449073, 94662073, 75720..."
9,10,"[7513050, 89421073, 93992073, 93886073, 756005..."


In [21]:
popularity = final_df['news_id'].value_counts().reset_index()
popularity.columns = ['news_id', 'count']

In [22]:
popularity

,news_id,count
0,7557050,71
1,7575050,69
2,7536050,69
3,94061073,69
4,7552050,67
...,...,...
5804,33658073,1
5805,84075073,1
5806,75379073,1
5807,87783073,1


In [23]:
recs_pop = list(popularity.head(7).news_id)
result['popular_recommendation'] = result['user_id'].apply(lambda x: recs_pop)
result.head(5)

,user_id,actual,popular_recommendation
0,1,"[94006073, 94000073, 94062073, 94063073, 93893...","[7557050, 7575050, 7536050, 94061073, 7552050,..."
1,2,"[94006073, 94662073, 94679073, 94115073, 92326...","[7557050, 7575050, 7536050, 94061073, 7552050,..."
2,3,"[94006073, 94108073, 94057073, 94290073, 75470...","[7557050, 7575050, 7536050, 94061073, 7552050,..."
3,4,"[94647073, 94449073, 94669073, 94679073, 75730...","[7557050, 7575050, 7536050, 94061073, 7552050,..."
4,5,"[94000073, 94098073, 94366073, 94483073, 94381...","[7557050, 7575050, 7536050, 94061073, 7552050,..."


In [24]:
user_item_matrix = pd.pivot_table(final_df, 
                                  index='user_id', columns='news_id', 
                                  values='age_param', 
                                  fill_value=0                                       
                                 )
 
user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).T.tocsr()

user_item_matrix.head(5)

news_id  179050    1221050   1261050   1319050   1918050   1931050   1988050   \
user_id                                                                         
1        0.000000       0.0       0.0       0.0       0.0       0.0       0.0   
2        2.696872       0.0       0.0       0.0       0.0       0.0       0.0   
3        0.000000       0.0       0.0       0.0       0.0       0.0       0.0   
4        0.000000       0.0       0.0       0.0       0.0       0.0       0.0   
5        0.000000       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  1996050   2040050   2232050   2255050   2317050   2324050   2351073   \
user_id                                                                         
1             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5             0.0       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  2354050   2546050   2591050   2619050   2680050   2694073   2972050   \
user_id                                                                         
1             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5             0.0       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  3135050   3192050   3192073   3220050   3221050   3239050   3270050   \
user_id                                                                         
1             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5             0.0       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  3280050   3300050   3301050   3334050   3369050   3463050   3464073   \
user_id                                                                         
1             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5             0.0       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  3542050   3593050   3707050   3794050   3854050   3943050   4131050   \
user_id                                                                         
1             0.0       0.0       0.0       0.0  0.000000       0.0       0.0   
2             0.0       0.0       0.0       0.0  0.000000       0.0       0.0   
3             0.0       0.0       0.0       0.0  0.000000       0.0       0.0   
4             0.0       0.0       0.0       0.0  0.000000       0.0       0.0   
5             0.0       0.0       0.0       0.0  5.865103       0.0       0.0   

news_id  4151050   4174050   4202050   4249050   4256050   4267050   4293050   \
user_id                                                                         
1             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5             0.0       0.0       0.0       0.0       0.0       0.0       0.0   

news_id  4300050   43140

### Item-item

In [25]:
%%time

model = ItemItemRecommender(K=20, num_threads=4) 
model.fit(csr_matrix(user_item_matrix).T,  
          show_progress=True)

recs = model.recommend(userid=2, user_items=sparse_user_item, N=20)


Wall time: 67.8 ms


In [26]:
model.recommend(1, sparse_user_item, N=20)

[(738, 6544.134039152219),
 (736, 6544.134039152219),
 (5807, 6544.134039152219),
 (5792, 6544.134039152219),
 (5796, 6544.134039152219),
 (5808, 6544.134039152219),
 (5756, 6544.134039152219),
 (5797, 6544.134039152219),
 (737, 6544.134039152219),
 (5802, 6544.134039152219),
 (733, 6425.149783894906),
 (5758, 6310.414966325354),
 (5761, 6310.414966325354),
 (5734, 6092.8144502451705),
 (5735, 6092.8144502451705),
 (5736, 6092.8144502451705),
 (5742, 6092.8144502451705),
 (5741, 6092.8144502451705),
 (5737, 6092.8144502451705),
 (5738, 6092.8144502451705)]

In [27]:
user_ids = list(user_item_matrix.index.values)
news_ids = list(user_item_matrix.columns.values)
def recommend_user(user_id, model):
    user_index = user_ids.index(user_id)
    recommendations = model.recommend(user_index, sparse_user_item, N=20)

    result = [news_ids[x[0]] for x in recommendations]
    return result

In [28]:
result['itemitem'] = result.apply(lambda x: recommend_user(x['user_id'], model), axis='columns')
result

,user_id,actual,popular_recommendation,itemitem
0,1,"[94006073, 94000073, 94062073, 94063073, 93893...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[94634073, 94529073, 94235073, 94417073, 94366..."
1,2,"[94006073, 94662073, 94679073, 94115073, 92326...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[7622050, 7619050, 95370073, 95323073, 9532907..."
2,3,"[94006073, 94108073, 94057073, 94290073, 75470...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[94986073, 94860073, 94765073, 95114073, 95203..."
3,4,"[94647073, 94449073, 94669073, 94679073, 75730...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[95281073, 95027073, 95212073, 95011073, 95152..."
4,5,"[94000073, 94098073, 94366073, 94483073, 94381...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[95075073, 95090073, 7592050, 7590050, 9501507..."
...,...,...,...,...
234,274,"[94098073, 94156073, 94289073, 94366073, 94556...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[7560050, 93992073, 94692073, 94849073, 947810..."
235,275,"[94480073, 94483073, 7572050, 7574050, 9446907...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[94898073, 95081073, 7583050, 95028073, 758505..."
236,276,"[94000073, 94113073, 94205073, 93978073, 93867...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[7592050, 7589050, 7588050, 7587050, 7593050, ..."
237,277,"[94000073, 94113073, 93315073, 93979073, 93695...","[7557050, 7575050, 7536050, 94061073, 7552050,...","[94692073, 94849073, 94781073, 94765073, 94730..."
